In [1]:
%load_ext autoreload
%autoreload 2

In [24]:
import sys
import numpy as np
from sklearn.metrics import roc_auc_score
import pandas as pd
#project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
#sys.path.insert(0, project_root)
sys.path.append("..") 
sys.path.append("../src")
from src.load_data import  prepare_volatility_data
from src.load_data import load_pivoted_field
from src.forecast import build_forecast_pipeline
from src.signals import generate_signals_and_positions
from src.analytic_utils import align_multiple
from src.checkpoint_storage import save_all_checkpoints_v2,load_all_checkpoints_v2
from src.ledger_portfolio import TradeLedger










In [6]:
checkpoint=load_all_checkpoints_v2("../checkpoints/har_pre_model_20250804_140707")
print(type(checkpoint), checkpoint.keys() if isinstance(checkpoint, dict) else None)
ds_test=checkpoint["test"]

print(type(ds_test), ds_test.keys() if isinstance(ds_test, dict) else None)
print(checkpoint["train"]["X_df"].columns.tolist())

best_model=checkpoint["model"]
print(best_model.feature_names_in_)


idx_df=ds_test['idx']  
multi_idx = pd.MultiIndex.from_frame(idx_df, names=["timestamp", "ticker"])
multi_idx

<class 'dict'> dict_keys(['train', 'val', 'test', 'model', 'metrics', 'config', 'notes'])
<class 'dict'> dict_keys(['X_df', 'y_series', 'label_matrix', 'idx', 'features'])
['signal', 'signal_rank', 'implied', 'implied_rank']
['signal' 'signal_rank' 'implied' 'implied_rank']


MultiIndex([('2023-11-20',  'AAL'),
            ('2023-11-20',  'AAP'),
            ('2023-11-20', 'AAPL'),
            ('2023-11-20', 'ABBV'),
            ('2023-11-20',  'ABT'),
            ('2023-11-20',  'ACN'),
            ('2023-11-20', 'ADBE'),
            ('2023-11-20',  'ADM'),
            ('2023-11-20',  'ADP'),
            ('2023-11-20',  'AEP'),
            ...
            ('2023-12-29',  'WRB'),
            ('2023-12-29',  'WRK'),
            ('2023-12-29',  'WST'),
            ('2023-12-29',   'WY'),
            ('2023-12-29', 'WYNN'),
            ('2023-12-29',  'XOM'),
            ('2023-12-29',  'ZBH'),
            ('2023-12-29', 'ZBRA'),
            ('2023-12-29', 'ZION'),
            ('2023-12-29',  'ZTS')],
           names=['timestamp', 'ticker'], length=11020)

In [5]:
X_test=checkpoint["test"]["X_df"]
y_test=checkpoint["test"]["y_series"]
y_pred=best_model.predict_proba(X_test)[:,1]
y_pred_binary=np.where(y_pred>0.5,1,0)
y_test_binary=np.where(y_test>0.5,1,0)
accuracy=roc_auc_score(y_test_binary,y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7474987841253247


In [7]:
y_pred_series = pd.Series(y_pred, index=multi_idx)
y_pred_series
y_pred_matrix=y_pred_series.unstack()
y_pred_matrix
ranked = y_pred_matrix.rank(axis=1, pct=True)
ranked
buy_mask = ranked >= 0.9
sell_mask = ranked <= 0.1
buy_mask
trades = []

for dt in y_pred_matrix.index:
    for ticker in y_pred_matrix.columns:
        action = None
        if buy_mask.loc[dt, ticker]:
            action = "buy"
        elif sell_mask.loc[dt, ticker]:
            action = "sell"
        if action:
            trades.append((dt, ticker, action))

In [10]:
import json
import yaml

with open("../configs/config.json") as f:
    config_data = json.load(f)

with open("../configs/config.yaml") as f:
    config_exp = yaml.safe_load(f)

print(config_data)
print(config_exp)
vol_data = prepare_volatility_data(config_data)

{'realized_path': 'data/iv_data.parquet', 'implied_path': 'data/iv_implied.parquet', 'price_path': 'data/underlying_prices.parquet', 'forecast_horizon': 5, 'model_path': 'models/best_model.pkl', 'new_realized_path': 'data/new_realized.parquet', 'new_implied_path': 'data/new_implied.parquet', 'option_chain_path': 'data/option_chain.parquet', 'output_ledger_path': 'results/ledger.parquet', 'train_test_split_ratio': 0.7, 'train_test_gap_days': 21, 'ranking_threshold': 0.9, 'signal_z': True, 'implied_z': True, 'rf_params': {'n_estimators': 100, 'max_depth': 5}, 'har_windows': {'short': 1, 'medium': 5, 'long': 21}}
{'experiment_name': 'v2_rf_20pct', 'signal_threshold': 0.2, 'features': ['signal', 'signal_rank', 'implied', 'implied_rank'], 'model': {'type': 'random_forest', 'hyperparams': {'n_estimators': 100, 'max_depth': 5}}}


/Users/stirlitz/ncGitHub/vol_forecast/notebooks/../src/load_data.py:149: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_returns = df_prices.pct_change()


In [11]:
close_matrix=load_pivoted_field("close")
close_matrix.index.rename("date", inplace=True)
close_matrix.head()

ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACGL,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,156.420,18.7400,236.76,182.000,135.42,132.65,139.04,44.550,407.20,564.37,...,87.75,67.95,63.545,56.900,116.90,136.56,129.13,583.89,64.270,234.01
2022-01-04,151.200,19.0200,237.16,179.690,135.14,131.40,135.76,45.130,404.48,554.10,...,87.49,68.37,65.910,57.190,118.10,137.31,130.44,587.06,66.055,225.04
2022-01-05,148.680,18.6900,236.45,174.835,135.88,132.50,135.17,44.595,397.25,514.28,...,84.25,69.05,66.740,56.270,116.57,135.58,130.01,558.65,66.200,216.65
2022-01-06,149.110,18.5653,241.59,172.010,135.20,130.45,135.14,44.855,378.01,514.05,...,84.64,68.52,68.310,56.210,116.36,136.90,128.92,555.16,68.930,217.46
2022-01-07,145.142,19.2900,238.09,172.220,134.88,133.12,135.54,45.080,370.75,510.63,...,84.53,69.12,68.880,56.405,115.39,135.37,128.14,530.57,70.080,211.09


In [15]:
def get_daily_snapshot(
    date: pd.Timestamp,
    close_matrix: pd.DataFrame,
    iv_matrix: pd.DataFrame
) -> pd.DataFrame:
    """
    Return a DataFrame with 'ticker', 'close', 'implied_vol' for a given date.
    """
    close_row = close_matrix.loc[date]
    iv_row = iv_matrix.loc[date]

    common_tickers = close_row.index.intersection(iv_row.index)

    df = pd.DataFrame({
        "ticker": common_tickers,
        "close": close_row[common_tickers].values,
        "implied_vol": iv_row[common_tickers].values
    }).dropna()

    return df

df=get_daily_snapshot(pd.Timestamp("2022-07-28"), close_matrix, vol_data["implied"])

In [16]:
import numpy as np
import pandas as pd
import py_vollib.black_scholes
import py_vollib_vectorized



def compute_option_price(spot, strike, cp, ttm, r, q, sigma):
    """
    Vectorized option pricing using py_vollib_vectorized.black_scholes.
    
    Parameters:
        spot: float or array
        strike: float or array
        cp: 'call' or 'put' or array-like
        ttm: float or array (in years)
        r: risk-free rate (scalar or array)
        q: dividend yield (scalar or array)
        sigma: implied volatility (float or array)
    
    Returns:
        np.ndarray of option prices
    """
    cp = np.asarray(cp)
    print(f"cp: {cp}")
    cp = np.where(cp == 'call', 'c', 'p')  # convert to py_vollib format
    print(f"cp: {cp}")

    return py_vollib.black_scholes.black_scholes(
        flag=cp,
        S=np.asarray(spot),
        K=np.asarray(strike),
        t=np.asarray(ttm),
        r=np.asarray(r),
        sigma=np.asarray(sigma)
    )

In [18]:
import numpy as np
import pandas as pd
import py_vollib.black_scholes
import py_vollib_vectorized



def compute_option_price(spot, strike, cp, ttm, r, q, sigma):
    """
    Vectorized option pricing using py_vollib_vectorized.black_scholes.
    
    Parameters:
        spot: float or array
        strike: float or array
        cp: 'call' or 'put' or array-like
        ttm: float or array (in years)
        r: risk-free rate (scalar or array)
        q: dividend yield (scalar or array)
        sigma: implied volatility (float or array)
    
    Returns:
        np.ndarray of option prices
    """
    cp = np.asarray(cp)
    cp = np.where(cp == 'call', 'c', 'p')  # convert to py_vollib format

    #print(f"spot: {spot}")
    #print(f"strike: {strike}")
    #print(f"cp: {cp}")
    #print(f"ttm: {ttm}")
    #print(f"r: {r}")
    #print(f"q: {q}")
    #print(f"sigma: {sigma}")
    return py_vollib.black_scholes.black_scholes(
        flag=cp,
        S=np.asarray(spot),
        K=np.asarray(strike),
        t=np.asarray(ttm),
        r=np.asarray(r),
        sigma=np.asarray(sigma),
        return_as='numpy'
    )

In [20]:
def expand_and_record_straddle_trades(trades, ledger, close_matrix, iv_matrix):
    print(f'trades: {trades}')
    vols= iv_matrix.loc[trades["timestamp"], trades["underlying"]]
    spots= close_matrix.loc[trades["timestamp"], trades["underlying"]]

    strike = round(spot, 1)  # optional: round to nearest 0.5 or 1
    expiry = dt + timedelta(days=1)
    ttm = 1/252  # or actual days to expiry

    for opt_type in ['call', 'put']:
        price = compute_option_price(
            spot=spot,
            strike=strike,
            cp=opt_type,
            ttm=ttm,
            r=0.0,
            q=0.0,
            sigma=vols
        )

        ledger.record_trade(
            timestamp=dt,
            action=action,
            instrument_type="option",
            underlying=ticker,
            option_type=opt_type,
            strike=strike,
            expiry=expiry.date(),
            quantity=1,
            price=price
        )


In [21]:
from datetime import timedelta

def generate_straddle_trades(
    trades,              # List of (timestamp, ticker, action)
    close_matrix,        # DataFrame: [date x ticker] of spot prices
    vol_matrix,          # DataFrame: [date x ticker] of implied vols
    ledger,              # Your TradeLedger instance
    ttm_days=5,          # Days to expiry
    r=0.0,
    q=0.0
):
    trade_dates = [t[0] for t in trades]
    time_stamps = pd.to_datetime(trade_dates).tz_localize("America/New_York")
    #print (f"trade_dates: {trade_dates}")
    tickers = [t[1] for t in trades]
    actions = [t[2] for t in trades]
    notionals = np.array([1 if a == "buy" else -1 for a in actions])

    spots = np.array([
        close_matrix.loc[dt, ticker] for dt, ticker in zip(trade_dates, tickers)
    ])
    vols = np.array([
        vol_matrix.loc[dt, ticker] for dt, ticker in zip(trade_dates, tickers)
    ])
    strike = spots
    qty = 1 / spots
    signed_qty = qty * notionals
    #print (f"about to compute expiry")
    expiry = [dt + timedelta(days=ttm_days) for dt in trade_dates]
    #print (f"expiry: {expiry}")
    ttm = np.full(len(spots), ttm_days / 252)
    r_arr = np.full(len(spots), r)
    q_arr = np.full(len(spots), q)

    for opt_type in ["call", "put"]:
        price = compute_option_price(
            spot=spots,
            strike=strike,
            cp=[opt_type] * len(spots),
            ttm=ttm,
            r=r_arr,
            q=q_arr,
            sigma=vols
        )

        trades_df = pd.DataFrame({
            "timestamp": time_stamps,
            "action": actions,
            "instrument_type": ["option"] * len(trade_dates),
            "underlying": tickers,
            "option_type": [opt_type] * len(trade_dates),
            "strike": strike,
            "expiry": expiry,
            "quantity": qty,
            "price": price,
            "signed_quantity": signed_qty,
            "total_cost": price * signed_qty
        })

        ledger.record_trades(trades_df)

In [25]:
ledger=TradeLedger()
generate_straddle_trades(trades, close_matrix, vol_data["implied"],ledger)

/Users/stirlitz/ncGitHub/vol_forecast/notebooks/../src/ledger_portfolio.py:230: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.trades_df = pd.concat([self.trades_df, normalized], ignore_index=True)


In [27]:
ledger.trades












,timestamp,action,instrument_type,underlying,option_type,strike,expiry,quantity,price,signed_quantity,total_cost,trade_id
0,2023-11-20 05:00:00+00:00,sell,option,aal,call,12.390,2023-11-25,0.080710,0.222394,-0.080710,-0.017949,253fb3977c4dab1fbc76696e
1,2023-11-20 05:00:00+00:00,sell,option,czr,put,46.260,2023-11-25,0.021617,0.933517,-0.021617,-0.020180,edf3d4979864282abd9374f1
2,2023-11-20 05:00:00+00:00,buy,option,dd,put,71.380,2023-11-25,0.014010,0.586937,0.014010,0.008223,324cc522b637f18590507ae8
3,2023-11-20 05:00:00+00:00,sell,option,dish,put,3.655,2023-11-25,0.273598,0.167796,-0.273598,-0.045908,01546506641eda43d74d9b3e
4,2023-11-20 05:00:00+00:00,sell,option,dxcm,put,109.010,2023-11-25,0.009173,2.194505,-0.009173,-0.020131,20361fb31e6964fd50de352f
...,...,...,...,...,...,...,...,...,...,...,...,...
4415,2023-12-29 05:00:00+00:00,buy,option,ndsn,call,264.190,2024-01-03,0.003785,2.321308,0.003785,0.008787,2bb2810c18409337ec8120e0
4416,2023-12-29 05:00:00+00:00,buy,option,ntap,call,88.150,2024-01-03,0.011344,0.784142,0.011344,0.008896,cf18b8ce37ee30f1ce5a107d
4417,2023-12-29 05:00:00+00:00,sell,option,ntrs,call,84.380,2024-01-03,0.011851,1.527488,-0.011851,-0.018102,aac66dc827b378b1705f97da
4418,2023-12-29 05:00:00+00:00,buy,option,hon,call,209.730,2024-01-03,0.004768,1.567545,0.004768,0.007474,a023cb7ce4ea159b70b7fd8b
